# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SCOPA_Sex_Processed&Results_Files
df = pd.read_csv('SexFemaleCateg_Methylome_APPRDX.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SCOPA_Sex_Processed&Results_Files
(150, 459)


,Sentrix,PATNO,HYS,Gender,APPRDX,SCOPA_Sex,cg06168319,cg20161984,cg03497431,cg19050973,...,cg08830117,cg00289134,cg18304052,cg22552004,cg16216706,cg21146019,cg06720722,cg26251795,cg15535628,cg04860563
0,200991620021_R03C01,3002,2,2,1,1,0.011066,0.178036,0.015360,0.026076,...,0.844128,0.884618,0.877310,0.850813,0.751936,0.834262,0.782225,0.938638,0.902261,0.925377
1,200989060105_R03C01,3003,2,2,1,0,0.009795,0.236185,0.016424,0.043095,...,0.876508,0.883615,0.845442,0.788270,0.794989,0.805975,0.676839,0.941932,0.887182,0.922311
2,200989060162_R04C01,3006,2,2,1,1,0.007707,0.083531,0.014420,0.024824,...,0.879566,0.874569,0.887354,0.834318,0.679218,0.839051,0.844670,0.917276,0.891738,0.920081
3,200973410121_R04C01,3018,2,2,1,1,0.008595,0.141270,0.017323,0.030682,...,0.837663,0.783088,0.801432,0.701147,0.759155,0.754705,0.848622,0.923401,0.778501,0.887822
4,201004820204_R01C01,3020,4,2,1,0,0.010678,0.208834,0.018890,0.041266,...,0.858647,0.839440,0.812828,0.728419,0.758121,0.787148,0.787153,0.934701,0.811828,0.890930


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'SCOPA_Sex'], axis=1)
df1.head(2)

,APPRDX,cg06168319,cg20161984,cg03497431,cg19050973,cg13736879,cg24939380,cg26438635,cg10657156,cg16947947,...,cg08830117,cg00289134,cg18304052,cg22552004,cg16216706,cg21146019,cg06720722,cg26251795,cg15535628,cg04860563
0,1,0.011066,0.178036,0.015360,0.026076,0.595214,0.854761,0.083951,0.017483,0.026387,...,0.844128,0.884618,0.877310,0.850813,0.751936,0.834262,0.782225,0.938638,0.902261,0.925377
1,1,0.009795,0.236185,0.016424,0.043095,0.609641,0.825202,0.099324,0.029517,0.028967,...,0.876508,0.883615,0.845442,0.788270,0.794989,0.805975,0.676839,0.941932,0.887182,0.922311


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000,...,2.000000,2.0000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
cg06168319,0.011066,0.009795,0.007707,0.008595,0.010678,0.009985,0.008823,0.008926,0.0117,0.010325,...,0.010064,0.0089,0.011013,0.010743,0.007761,0.007166,0.011383,0.009672,0.007513,0.009425


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,...,2.000000,2.0000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,cg06168319,0.011066,0.009795,0.007707,0.008595,0.010678,0.009985,0.008823,0.008926,0.0117,...,0.010064,0.0089,0.011013,0.010743,0.007761,0.007166,0.011383,0.009672,0.007513,0.009425


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg06168319,0.011066,0.009795,0.007707,0.008595,0.010678,0.009985,0.008823,0.008926,0.011700,...,0.010064,0.008900,0.011013,0.010743,0.007761,0.007166,0.011383,0.009672,0.007513,0.009425
2,cg20161984,0.178036,0.236185,0.083531,0.141270,0.208834,0.195296,0.088224,0.111894,0.208542,...,0.190479,0.207007,0.154997,0.168180,0.224789,0.051830,0.152380,0.213348,0.207330,0.215902


In [8]:
df2 = pd.read_csv ("SexFemale_Boruta_Top50.csv")
df2.head(2)

,CpG,Ranking
0,APPRDX,1
1,cg14773441,1


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Ranking
0,cg13736879,0.595214,0.609641,0.639893,0.660239,0.494526,0.622538,0.587471,0.558542,0.613746,...,0.665221,0.550306,0.621918,0.565457,0.555443,0.607909,0.675731,0.630617,0.649625,6
1,cg26438635,0.083951,0.099324,0.081497,0.110557,0.079881,0.078058,0.112834,0.103053,0.076918,...,0.062129,0.092509,0.073045,0.120304,0.066443,0.103939,0.079775,0.093707,0.072194,45


In [10]:
df3 = df3.drop(['Ranking'], axis=1)
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg13736879,0.595214,0.609641,0.639893,0.660239,0.494526,0.622538,0.587471,0.558542,0.613746,...,0.659782,0.665221,0.550306,0.621918,0.565457,0.555443,0.607909,0.675731,0.630617,0.649625
1,cg26438635,0.083951,0.099324,0.081497,0.110557,0.079881,0.078058,0.112834,0.103053,0.076918,...,0.075909,0.062129,0.092509,0.073045,0.120304,0.066443,0.103939,0.079775,0.093707,0.072194


In [11]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,cg08315265,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
1.0,0.595214,0.083951,0.017483,0.026387,0.893037,0.018019,0.945777,0.908718,0.939624,0.889103,...,0.853369,0.877772,0.093989,0.877626,0.828106,0.898532,0.853922,0.853745,0.875538,0.850813


In [12]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [13]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
0,CpG,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
1,1.0,0.595214,0.083951,0.017483,0.026387,0.893037,0.018019,0.945777,0.908718,0.939624,...,0.853369,0.877772,0.093989,0.877626,0.828106,0.898532,0.853922,0.853745,0.875538,0.850813


In [14]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
1,1.0,0.595214,0.083951,0.017483,0.026387,0.893037,0.018019,0.945777,0.908718,0.939624,...,0.853369,0.877772,0.093989,0.877626,0.828106,0.898532,0.853922,0.853745,0.875538,0.850813
2,1.0,0.609641,0.099324,0.029517,0.028967,0.876945,0.023695,0.96087,0.858124,0.930102,...,0.805801,0.852513,0.118786,0.902322,0.844675,0.888375,0.83717,0.827285,0.881061,0.78827


In [15]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
0,1.0,0.595214,0.083951,0.017483,0.026387,0.893037,0.018019,0.945777,0.908718,0.939624,...,0.853369,0.877772,0.093989,0.877626,0.828106,0.898532,0.853922,0.853745,0.875538,0.850813
1,1.0,0.609641,0.099324,0.029517,0.028967,0.876945,0.023695,0.96087,0.858124,0.930102,...,0.805801,0.852513,0.118786,0.902322,0.844675,0.888375,0.83717,0.827285,0.881061,0.78827
2,1.0,0.639893,0.081497,0.020152,0.023298,0.883127,0.019443,0.963261,0.924197,0.930916,...,0.877537,0.87764,0.127289,0.848723,0.874952,0.880456,0.808825,0.804474,0.825043,0.834318
3,1.0,0.660239,0.110557,0.018114,0.022809,0.82003,0.01504,0.95375,0.903606,0.922441,...,0.784478,0.758163,0.134542,0.890995,0.870576,0.764592,0.788972,0.737888,0.835351,0.701147
4,1.0,0.494526,0.079881,0.020243,0.03195,0.846988,0.020309,0.936899,0.878429,0.875692,...,0.808769,0.812906,0.112292,0.873495,0.841886,0.842832,0.761205,0.732354,0.874457,0.728419


In [16]:
df3.shape

(150, 51)

In [17]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [18]:
df3.head(2)

,APPRDX,cg13736879,cg26438635,cg10657156,cg16947947,cg11798066,cg00798428,cg08163494,cg22353381,cg09201719,...,cg22889925,cg04513561,cg15007954,cg25708695,cg00306305,cg22662645,cg09085792,cg25756974,cg12848971,cg22552004
0,1,0.595214,0.083951,0.017483,0.026387,0.893037,0.018019,0.945777,0.908718,0.939624,...,0.853369,0.877772,0.093989,0.877626,0.828106,0.898532,0.853922,0.853745,0.875538,0.850813
1,1,0.609641,0.099324,0.029517,0.028967,0.876945,0.023695,0.96087,0.858124,0.930102,...,0.805801,0.852513,0.118786,0.902322,0.844675,0.888375,0.83717,0.827285,0.881061,0.78827


In [19]:
df3.shape

(150, 51)

In [21]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [22]:
dframe1.to_csv ("SexFemale_Top50CpG_Accuracy_1000RF.csv")

## RanFor 1000 times, CpG with Severity (Normal, Mild, Severe) 

In [23]:
df4 = df3
df4 ['SCOPA_Sex'] = df ['SCOPA_Sex'].apply(int)

In [24]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df4.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
dframe2.head()

,Accuracy,Std_Dev
0,0.628571,0.171429
1,0.614286,0.091473
2,0.514286,0.171429
3,0.628571,0.159079
4,0.514286,0.193781


In [26]:
dframe2.to_csv ("SexFemale_Top50CpG_Severity_Accuracy_1000RF.csv")